In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import Imagenette
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

# Function to calculate the output size after convolutional and pooling layers
def calculate_output_size(input_size, conv_layers, pool_layers):
    output_size = input_size
    for conv_layer in conv_layers:
        output_size = (output_size - conv_layer.kernel_size[0] + 2 * conv_layer.padding[0]) // conv_layer.stride[0] + 1
    for pool_layer in pool_layers:
        output_size = (output_size - pool_layer.kernel_size) // pool_layer.stride + 1
    return output_size

# Prepare the dataset
train_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

train_dataset = Imagenette("data/imagenette/train/", split="train", size="160px", download=True, transform=train_transforms)

# Use 10% of the training set for validation
train_set_size = int(len(train_dataset) * 0.9)
val_set_size = len(train_dataset) - train_set_size

seed = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_set_size, val_set_size], generator=seed)

# Use DataLoader to load the dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, num_workers=8, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, num_workers=8, shuffle=False)


# Configure the test dataset
test_dataset = Imagenette("data/imagenette/test/", split="val", size="160px", download=True, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, num_workers=8, shuffle=False)



class BasicCNN(pl.LightningModule):
    def __init__(self, num_classes=10):
        super().__init__()

        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Define pooling layers
        self.pool = nn.MaxPool2d(2, 2)

        # Calculate output size for fully connected layer
        input_size = (64, 64)
        output_size = calculate_output_size(input_size[0], [self.conv1, self.conv2, self.conv3], [self.pool, self.pool, self.pool])
        fc1_input_size = 64 * output_size * output_size

        # Define fully connected layers
        self.fc1 = nn.Linear(fc1_input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

        self.train_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.val_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.test_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the output tensor before passing to the fully connected layer
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss)

        # Compute and log training accuracy
        pred_labels = torch.argmax(y_hat, dim=1)
        acc = self.train_accuracy(pred_labels, y)
        self.log("train_accuracy", acc, on_step=False, on_epoch=True)

        return loss



    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.val_accuracy(y_hat, y)
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log("val_accuracy", self.val_accuracy, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.test_accuracy(y_hat, y)
        self.log("test_loss", loss, on_step=False, on_epoch=True)
        self.log("test_accuracy", self.test_accuracy, on_step=False, on_epoch=True)



    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def on_train_epoch_end(self):
        self.train_accuracy.reset()


# Model instantiation
model = BasicCNN()

# Early stopping callback
early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5)

# Model checkpoint callback
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")

# Fit the model
trainer = pl.Trainer(callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model, train_loader, val_loader)


100%|██████████| 99003388/99003388 [00:02<00:00, 38976830.68it/s]


Extracting data/imagenette/train/imagenette2-160.tgz to data/imagenette/train/


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


100%|██████████| 99003388/99003388 [00:02<00:00, 40693603.34it/s]


Extracting data/imagenette/test/imagenette2-160.tgz to data/imagenette/test/


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [6]:
print(trainer.logged_metrics)

{'train_loss': tensor(0.2747), 'val_loss': tensor(1.1642), 'val_accuracy': tensor(0.6441), 'train_accuracy': tensor(0.8344)}


In [8]:
Train_metrics = trainer.logged_metrics

# Evaluation
print("Train Metrics")
print("Train Accuracy:", Train_metrics["train_accuracy"])
print("Train Loss:", Train_metrics["train_loss"])
print("Validation Accuracy:", Train_metrics["val_accuracy"])
print("Validation Loss:", Train_metrics["val_loss"])

Train Metrics
Train Accuracy: tensor(0.8344)
Train Loss: tensor(0.2747)
Validation Accuracy: tensor(0.6441)
Validation Loss: tensor(1.1642)


In [9]:
trainer.test(model, test_loader)
Test_metrics = trainer.logged_metrics
# Evaluation
print("Test Metrics")
print("Test Accuracy:", Test_metrics["test_accuracy"])
print("Test Loss:", Test_metrics["test_loss"])

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6267516016960144     │
│         test_loss         │    1.2767493724822998     │
└───────────────────────────┴───────────────────────────┘

Test Metrics
Test Accuracy: tensor(0.6268)
Test Loss: tensor(1.2767)


In [10]:
#Saving the Basic CNN model
torch.save(model.state_dict(), 'Basic_CNN_Model.pth')